In [16]:
import sys, codecs
output_file = codecs.open("33_EN_NER-formatted.tsv", 'w', 'utf-8')
input_file  = codecs.open("33_English_05_07_2021.json", 'r', 'utf-8')
import json

print ("String (Title or Query)","\t","Token","\t","Tag","\t","Span Token","\t","Span Tag",file = output_file)
with input_file as f:
    jsondata = json.load(f)
for each_segment in jsondata:
    full_segment = each_segment['text']
    seg_tokens = each_segment['tokens']
    seg_tags = each_segment['tags']
    
    spans = []
    span_tag_index = None
    previous_span_tag_index = None
    for token in seg_tokens:
        for idx, tag in enumerate(seg_tags):
            if (token['start'] >= tag['start']) and (token['end'] <= tag['end']):
                if token['start']>tag['start']:
                    formattag = "BLANK"
                else:
                    formattag = token['tag']
                span_tag_index = idx
                span_tag_text = tag['text']
                span_tag_label = tag['label']
                print (full_segment,"\t",token['text'],"\t",token['tag'],"\t",token['text'],"\t",formattag,file = output_file)
        if previous_span_tag_index is None:
            if span_tag_index is not None:
                #First tag found for token init span
                span = {}
                span['tokens'] = []
                span['tokens'].append(token['tokenIndex'])
                span['label'] = span_tag_label
                span['text'] = span_tag_text
        else:
            if previous_span_tag_index == span_tag_index:
                #Continued span, log token
                span['tokens'].append(token['tokenIndex'])
            else:
                #New tag append current and start new span
                spans.append(span)
                span ={}
                span['tokens']=[]
                span['tokens'].append(token['tokenIndex'])
                span['label'] = span_tag_label
                span['text'] = span_tag_text
        previous_span_tag_index = span_tag_index

    spans.append(span)
output_file.close()
input_file.close()

